In [2]:
from os import listdir
from os.path import isfile, join, isdir
from lxml import etree
from time import time
from shapely.geometry import box
from shapely.ops import cascaded_union
import pdftableextract as pte
import numpy as np
import pdfquery
import json
import ast
import pandas as pd
import numpy as np
import pdfquery

import os
import re
from math import floor
from pymongo import MongoClient
class dbConf(object):
    # connect to database
    client = MongoClient('mongodb://localhost:27017/')
    try:
        mdb = client.racmo
        Notifications = mdb.Notifications
        PdfFiles=mdb.PdfFiles
        Keywords=mdb.Keywords
        Ticket=mdb.Ticket
        Documents=mdb.Documents
    except Exception as e:
        print(e)


In [2]:
PDF_DIR = "/home/thrymr/Racmo/RacmoIT/process/Gestured documents 15-02-2018"


In [11]:
fl = [f for f in listdir(PDF_DIR)]

In [135]:

se={f.split(".")[-1].lower() for f in fl}
d=dict()
for s in se:
    d.update({s:0})

# for f in fl

In [3]:
UPDIR="/home/thrymr/Racmo/RacmoIT"
def df_to_json(df):
    js = {}
    vals = df[0].values
    ls = []
    for i, v in enumerate(vals):
        if not v:
            ls.append(ls[-1])
        else:
            ls.append(v)
    df[0] = ls
    for key, idx in df.groupby([0]).groups.items():
        
        js[key] = {}
        
        for val in df.loc[idx][[1,2]].values:
            if val[0] == "" and val[1] == "":
                js[key]["_value"] = ""
            elif val[0] == "" and val[1] != "":
                js[key]["_value"] = val[1]
            elif val[0] != "" and val[1] == "":
                js[key]["_value"] = val[0]
            elif val[0] != "" and val[1] != "":
                js[key][val[0]] = val[1]                
        if(key=="Documentos"):
            js[key] = df.loc[idx][[1,2]].values
        if len(js[key]) == 1 and "_value" in js[key]:
            js[key] = js[key]["_value"]
    return js
def dump_json():
    _JSON = {}  
    pdf_files = [f for f in listdir(PDF_DIR)\
                 if isfile(join(PDF_DIR,  f)) and 
                 f[-3:].lower() == "pdf" ]
    
    for pdf_file in pdf_files:#
        pdf_path = join(PDF_DIR, pdf_file)
        _JSON[pdf_file] = {"filepath" : pdf_path}
        try:
            pdf = pdfquery.PDFQuery(pdf_path)
            pdf.load()
            root = pdf.tree.getroot().getchildren()[0]
            page_box = [float(x) for x in root.get("bbox")[1:-1].split(",")]
            tables, _ =\
            zip(
                *sorted(
                    [(p.bounds,p.area) for p in cascaded_union(
                        [box(*[float(x) for x in node.get("bbox")[1:-1].split(",")]) for node in root.iter() if node.tag == "LTRect"]
                    )],
                    key = lambda x : -x[1]
                )
            )
            X = page_box[2]
            Y = page_box[3]
            xf = 11.69/X
            yf = 8.27/Y
            t1, t2 = tables
            table_1_bbox = ":".join(map(str,(t1[0]*xf - 0.1, (Y - t1[3])*yf - 0.1, t1[2]*xf + 0.1, (Y - t1[1])*yf + 0.1)))
            table_2_bbox = ":".join(map(str,(t2[0]*xf - 0.1, (Y - t2[3])*yf - 0.1, t2[2]*xf + 0.1, (Y - t2[1])*yf + 0.1)))

            df1 =\
            pd.DataFrame(
                pte.table_to_list(
                    pte.process_page(
                        pdf_path,
                        "1",
                        crop = table_1_bbox,
                        pad=20
                    ),
                    "1"
                )[1]
            )

            _JSON[pdf_file]["table_1"] = df_to_json(df1)
            df2 = \
            pd.DataFrame(
                pte.table_to_list(
                    pte.process_page(
                        pdf_path,
                        "1",
                        crop = table_2_bbox,
                        pad=20
                    ),
                    "1"
                )[1]
            )
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            _JSON[pdf_file]["table_2"] = df2.to_json(orient='index')
            mdb = dbConf.mdb
            Ticket = dbConf.Ticket
            ti = {
                "filename": pdf_file,
                "JSON": _JSON[pdf_file]
            }
            Tickets.insert_one(ti)
        except Exception as e:
            print(e, pdf_path)
    
    with open(join(UPDIR,"all_tables_"+str(time())+".json"), "w") as f:
        f.write(json.dumps(_JSON))
    return _JSON


In [1]:
tickets=[i['filename'] for i in td]

NameError: name 'td' is not defined

In [1]:
tickets

In [4]:
def get_keyword(text):
    ndb = dbConf.mdb
    dd = ndb.get_collection('Keywords').find({})
    occu=dict()
    for j,it in enumerate(dd):
        c=0
        f=True
        for k in it['keyword']:
            if re.search(k, text, re.IGNORECASE):
                c+=1
            else:
                f=False
        if f and c>0:
            if(it['file_class']!='TI'):
                if it['file_class'] in occu:
                    occu[it['file_class']]['count']+=1
                    occu[it['file_class']]['keywords'].append(it['keyword'])
                else:
                    occu.update({it['file_class']:{'count':1,'keywords':[it['keyword']]}})
    return occu


In [315]:

mdb = dbConf.mdb
Ticket = dbConf.Ticket
# mdb.get_collection('Tickets').find({})
td=mdb.get_collection('Ticket').find({})


In [6]:
df =  pd.DataFrame(list(td))
del df['_id']
df['document']=df.apply(lambda x :{'document':x['JSON']['table_1']['Documentos']},axis=1)


In [307]:
def get_pdf_text_2(path):
    try:
        pdf=pdfquery.PDFQuery(path)
        pdf.load()
        pdftext=""
        pgn=len(pdf.tree.getroot().getchildren())
        for i in range(0,pgn):
            root = pdf.tree.getroot().getchildren()[i]
            for node in root.iter():
                try:
    #                 if node.tag == "LTTextLineHorizontal" or node.tag == "LTTextBox":
                    if node.text:
                        pdftext=pdftext+" "+node.text
    #                     pdftext=pdftext+"\n"
                except Exception as e:
                    print(node.tag, e)
        return pdftext
    except Exception as e:
        return('Error:'+str(e))


In [17]:
def get_pdf_text(path):
    pdf=pdfquery.PDFQuery(path)
    pdf.load()
    pdftext=""
    pgn=len(pdf.tree.getroot().getchildren())
    for i in range(0,pgn):
        root = pdf.tree.getroot().getchildren()[i]
        for node in root.iter():
            try:
                if node.tag == "LTTextLineHorizontal" or node.tag == "LTTextBox":
                    pdftext=pdftext+node.text
                    pdftext=pdftext+"\n"
            except Exception as e:
                print(node.tag, e)
    return pdftext
def get_rtf_text(path):
    print('unrtf --text '+path)
    text=os.popen('unrtf --text '+path).read()
    print(text)
    return text

In [166]:
tickets=list(set(tickets))
len(tickets)

989

In [68]:
df2=df[10:100]
s={len(row['JSON']['table_1']['Documentos']) for ind,row in df.iterrows()}
s        

{80, 95, 98, 99, 100, 101, 103, 105, 107, 109, 111, 120, 121, 124, 125}

In [116]:
principal_files=list()


In [125]:
for f in principal_files:
    d[ f.split(".")[-1].lower()]+=1
d

{'db': 0, 'json': 0, 'pdf': 499, 'png': 0, 'rtf': 508, 'zip': 0}

In [156]:
def noti_to_excel(principal_files):
    ls=list()
    for(key,v) in principal_files.items():
        print(key)
        for pf in v:
            text=""
            row={'Filename':pf,'Filetype':key}
            if pf[-3:].lower()=='pdf':
                text=get_pdf_text(join(PDF_DIR,pf))
            if pf[-3:].lower()=='rtf':
                text=os.popen('cd "'+PDF_DIR+'" && unrtf --text '+pf).read()
            occu=get_keyword(text)
            if occu:
                tlist,klist,c=zip(*[(k,v['keywords'],v['count']) for k,v in occu.items()])
                kl=""
                tl=""
                for tkeys in klist:
                    for keys in tkeys:
                        for k in keys:
                            kl=kl+k
                            kl=kl+"+"
                        kl=kl[:-1]
                        kl=kl+","
                        tl=tl+tlist[klist.index(tkeys)]+","
                kl=kl[:-1]
                tl=tl[:-1]
                row.update({'Keyword List':kl,'Keyword Type List':tl})
                ks=set(tlist)
                if(len(ks)==1):
                    row.update({'Unique':'Yes'})
                else:
                    row.update({'Unique':'No'})
       
            else:
                row.update({'Keyword List':'','Keyword Type List':'','Unique':'NA'})
            ls.append(row)
    exceldf=pd.DataFrame(ls)
    exceldf.describe()
    writer = pd.ExcelWriter('/home/thrymr/Result-Excel-'+str(time())+'.xlsx')
    exceldf.to_excel(writer,'Sheet1')
    writer.save()

In [157]:
docs={'Notification':list(pfiles),'Ticket':tickets}
noti_to_excel(docs)

Notification
Ticket


In [197]:
# len()
oth=[len(i) for i in filegroup.values()]

In [199]:
sum(oth)

167

In [214]:
pdf_files = [f for f in listdir(PDF_DIR)\
                 if isfile(join(PDF_DIR,  f)) ]
ls=list()    
for pdf_file in pdf_files:
    ls.append(pdf_file.split('_'))
df=pd.DataFrame(ls)

In [215]:
df.to_csv('/home/thrymr/file_dataframe.csv')

In [316]:
td

In [247]:
df=pd.read_csv('/home/thrymr/file_group.csv')

In [222]:
tickets=df['min_filename'].values

In [248]:
tmp = pd.DataFrame(columns=["file","group"])

In [249]:
j = 0;
for i, r in df.iterrows():
    for f in ast.literal_eval(r.files):
        tmp.loc[j] = [f, r.group]
        j = j+1

In [250]:
tmp.shape

(5951, 2)

In [251]:
def get_structured_files_dataframe(df):
    tmp = pd.DataFrame(columns=["file","group"])
    j = 0
    for i, r in df.iterrows():
        for f in ast.literal_eval(r.files):
            tmp.loc[j] = [f, r.group]
            j = j+1
    assert tmp.apply(lambda x:x.file.split("_")[3] == x.group, axis = 1).all()
    tmp["ext"] = tmp.apply(lambda x : x.file.split(".")[-1],axis =1)
    tmp["length"] = tmp.apply(lambda x : len(x.file),axis =1)
    TI = []
    OI = []
    for case, indices in tmp.groupby("group").groups.items():
        ticket_index = tmp.loc[indices].length.values.argmin()
        for i, idx in enumerate(indices):
            if i == ticket_index:
                TI.append(idx)
            else:
                OI.append(idx)

    tmp['type'] = "OTHER"
    tmp.loc[TI,"type"] = "TICKET"
    return tmp

df=pd.read_csv('/home/thrymr/file_group.csv')
temp = get_structured_files_dataframe(df)

In [20]:
def parse_ticket(pdf_path):
        _JSON = {"filepath" : pdf_path}
        try:
            pdf = pdfquery.PDFQuery(pdf_path)
            pdf.load()
            root = pdf.tree.getroot().getchildren()[0]
            page_box = [float(x) for x in root.get("bbox")[1:-1].split(",")]
            tables, _ =\
            zip(
                *sorted(
                    [(p.bounds,p.area) for p in cascaded_union(
                        [box(*[float(x) for x in node.get("bbox")[1:-1].split(",")]) for node in root.iter() if node.tag == "LTRect"]
                    )],
                    key = lambda x : -x[1]
                )
            )
            X = page_box[2]
            Y = page_box[3]
            xf = 11.69/X
            yf = 8.27/Y
            t1, t2 = tables
            table_1_bbox = ":".join(map(str,(t1[0]*xf - 0.1, (Y - t1[3])*yf - 0.1, t1[2]*xf + 0.1, (Y - t1[1])*yf + 0.1)))
            table_2_bbox = ":".join(map(str,(t2[0]*xf - 0.1, (Y - t2[3])*yf - 0.1, t2[2]*xf + 0.1, (Y - t2[1])*yf + 0.1)))

            df1 =\
            pd.DataFrame(
                pte.table_to_list(
                    pte.process_page(
                        pdf_path,
                        "1",
                        crop = table_1_bbox,
                        pad=20
                    ),
                    "1"
                )[1]
            )

            _JSON["table_1"] = df_to_json(df1)
            df2 = \
            pd.DataFrame(
                pte.table_to_list(
                    pte.process_page(
                        pdf_path,
                        "1",
                        crop = table_2_bbox,
                        pad=20
                    ),
                    "1"
                )[1]
            )
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            _JSON["table_2"] = df2.to_json(orient='index')
            mdb = dbConf.mdb
            Ticket = dbConf.Ticket
            ti = {
                "filename": pdf_file,
                "JSON": _JSON[pdf_file]
            }
        except Exception as e:
            return('Error:'+str(e))
    
        return str(_JSON)
    

def parse_other(pf):
    text=""
    if pf[-3:].lower()=='pdf':
                text=get_pdf_text_2(join(PDF_DIR,pf))
    if pf[-3:].lower()=='rtf':
                text=os.popen('cd "'+PDF_DIR+'" && unrtf --text '+pf).read()
    
    return text


for i, r in temp[5947:].iterrows():
    if r.type == "TICKET":
        report.loc[i,"response"] = parse_ticket(join(PDF_DIR,r.file))
    else:
        report.loc[i,"response"] = parse_other(r.file)        


In [18]:
def change_text(text):
    try:
        return text.encode('utf-8')
    except Exception as e:
        return text
report["response"] = report["response"].apply(change_text)

In [303]:
writer = pd.ExcelWriter('/home/thrymr/Report.xlsx')
report.to_excel(writer,'Sheet1')

UnicodeDecodeError: 'utf8' codec can't decode byte 0xf3 in position 370: invalid continuation byte

In [301]:
report=report.fillna('')


In [7]:
report=pd.read_csv('/home/thrymr/report.csv')

In [35]:
final_report=report[['file','group','type']]
for i,row in report.iterrows():
    row=row.fillna('')
    if row['file'] in prev_df['Filename'].values:
        
        final_report.loc[i,"keyword list"] = prev_df.loc[prev_df['Filename'] == row['file']]["Keyword List"].values[0]
        final_report.loc[i,"Keyword Type List"] = prev_df.loc[prev_df['Filename'] == row['file']]["Keyword Type List"].values[0]
        final_report.loc[i,"type"] = prev_df.loc[prev_df['Filename'] == row['file']]["Filetype"].values[0].upper()
        final_report.loc[i,"Unique"] = prev_df.loc[prev_df['Filename'] == row['file']]["Unique"].values[0]
        final_report.loc[i,"Status"]='Parsed'
    elif row['response'][:5]=='Error':
        final_report.loc[i,"Status"]='Cannot Parse'
    else:
        text=row['response']
        occu=get_keyword(text)
        r={}
        if occu:
                tlist,klist,c=zip(*[(k,v['keywords'],v['count']) for k,v in occu.items()])
                kl=""
                tl=""
                for tkeys in klist:
                    for keys in tkeys:
                        for k in keys:
                            kl=kl+k
                            kl=kl+"+"
                        kl=kl[:-1]
                        kl=kl+","
                        tl=tl+tlist[klist.index(tkeys)]+","
                kl=kl[:-1]
                tl=tl[:-1]
                r.update({'Keyword List':kl,'Keyword Type List':tl})
                ks=set(tlist)
                if(len(ks)==1):
                    r.update({'Unique':'Yes'})
                else:
                    r.update({'Unique':'No'})
       
        else:
                r.update({'Keyword List':'','Keyword Type List':'','Unique':'NA'})
        final_report.loc[i,"keyword list"] = r['Keyword List']
        final_report.loc[i,"Keyword Type List"] =r['Keyword Type List']
        final_report.loc[i,"Unique"] = r['Unique']
        final_report.loc[i,"Status"]='Parsed'
writer = pd.ExcelWriter('/home/thrymr/Final-Result-Excel-ALL-FILES.xlsx')
final_report.to_excel(writer,'Sheet1')
writer.save()
    

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio